In [171]:
import sqlite3
import pandas as pd
from soynlp.noun import LRNounExtractor_v2
from collections import defaultdict
import re

# Stopwords 처리
punc = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
pattern1 = re.compile(r'[{}]'.format(re.escape(punc))) # punctuation 제거
pattern2 = re.compile(r'[.]{2,}') # . 2개 이상이면, 1개로 바꾸기
pattern3 = re.compile(r'([ㄱ-ㅎㅏ-ㅣ])\1') # ㅋㅋ, ㄷㄷㄷ과 같은거 제거
pattern4 = re.compile(r'[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ. ]') # 특수문자 제거
pattern5 = re.compile(r'\s{2,}') # white space 1개로 바꾸기.

def cleaning(df):
    return pattern5.sub(' ', 
              pattern4.sub('',
               pattern3.sub('',
                pattern2.sub('.',
                 pattern1.sub('', df)))))

def extract_nouns(df):
    noun_extractor = LRNounExtractor_v2(verbose=True)
    nouns = noun_extractor.train_extract([df])
    words = defaultdict(lambda:0)
    for k, v in nouns.items():
        if len(k) > 1:
            words[k] = v
    return sorted(words.items(),key=lambda _:_[1], reverse=True)

def search_dict(nouns):
    # 사전 검색 결과 없는 단어 추출
    conn = sqlite3.connect('kr_korean.db')
    cur = conn.cursor()
    data = pd.read_sql('SELECT word FROM kr WHERE part="명사"', conn)
    data = ' '.join(data['word'])
    return pd.DataFrame([_ for _ in nouns if _[0] not in data])

In [172]:
conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head',conn)
df = ' '.join(df['head'])
df = cleaning(df)
nouns = extract_nouns(df)
new_word = search_dict(nouns)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 27791 from 1 sents. mem=0.478 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=55861, mem=0.477 Gb
[Noun Extractor] batch prediction was completed for 7035 words
[Noun Extractor] checked compounds. discovered 1139 compounds
[Noun Extractor] postprocessing detaching_features : 3172 -> 2914
[Noun Extractor] postprocessing ignore_features : 2914 -> 2861
[Noun Extractor] postprocessing ignore_NJ : 2861 -> 2852
[Noun Extractor] 2852 nouns (1139 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.477 Gb                    
[Noun Extractor] 45.96 % eojeols are covered
